In [5]:
import re, os, sys, json, time, requests
from pathlib import Path
import pandas as pd
import numpy as np

# TODO just load the parquets and run parse_eval
# or run my own eval

In [2]:
# TODO load config.json, load eval.parquet
fs = sorted(Path('../outputs').glob('**/eval.parquet'))
fs[-10:]

[PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_04-58-14/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_05-01-29/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_05-25-00/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_05-28-42/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_05-32-15/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-05_05-35-47/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-11_00-09-35/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook/2024-10-11_04-36-06/eval.parquet'),
 PosixPath('../outputs/wassname-llama-3-2-1b-sft_side-SV

In [9]:
def load_eval(f):
    print(f.parent.parent.stem)
    config = json.load((f.parent / 'config.json').open())
    log = (f.parent / 'log.txt').open().read()

    # dataframe of all non list/dict/tuple in config
    config_df = pd.DataFrame({k: v for k, v in config.items() if not isinstance(v, (list, dict, tuple))}, index=[0])
    config_df['log'] = log
    config_df['file'] = f
    config_df['ts'] = f.parent.stem
    _, method, _ = f.parent.parent.stem.split('_', 3) # this is {model}_{method}_{dataset}
    config_df['method'] = method

    # put key cols first
    key_columns = ['dataset', 'base_model'
    return config_df

df = pd.concat([load_eval(f) for f in fs[-10:]])
df


wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-MSE_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-PrefVec_us_history_textbook
wassname-llama-3-2-1b-sft_side-SVD-Rank_us_history_textbook


,dataset,verbose,dev,load_in_4bit,load_in_8bit,use_gradient_checkpointing,batch_size,n_samples,eval_samples,max_length,max_prompt_length,base_model,save,wandb,lr,collect_input,collect_hs,log,file,model
0,us_history_textbook,0,False,False,False,False,32,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000029,True,False,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_04-58-14
0,us_history_textbook,0,False,False,False,False,32,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000002,False,True,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_05-01-29
0,us_history_textbook,0,False,False,False,False,16,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000570,False,False,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_05-25-00
0,us_history_textbook,0,False,False,False,False,16,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000445,False,False,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_05-28-42
0,us_history_textbook,0,False,False,False,False,16,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000471,False,False,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_05-32-15
0,us_history_textbook,0,False,False,False,False,16,3600,128,196,96,wassname/llama-3-2-1b-sft,False,False,0.000553,False,False,,../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-05_05-35-47
0,us_history_textbook,1,False,False,False,False,16,3600,None,196,96,wassname/llama-3-2-1b-sft,True,True,0.000400,True,True,"October 11, 2024 > 00:09:38 | INFO | trainable...",../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-11_00-09-35
0,us_history_textbook,2,True,True,False,False,2,64,750,196,96,wassname/llama-3-2-1b-sft,True,False,0.000400,True,True,"October 11, 2024 > 04:36:12 | INFO | trainable...",../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-11_04-36-06
0,us_history_textbook,1,False,False,False,False,16,3600,None,196,96,wassname/llama-3-2-1b-sft,True,True,0.000400,True,True,"October 11, 2024 > 00:05:35 | INFO | trainable...",../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-11_00-05-32
0,us_history_textbook,1,False,False,False,False,16,3600,None,196,96,wassname/llama-3-2-1b-sft,True,True,0.000400,True,True,"October 11, 2024 > 00:01:19 | INFO | trainable...",../outputs/wassname-llama-3-2-1b-sft_side-SVD-...,2024-10-11_00-01-16


In [6]:
f = fs[-2]
print(f.parent.parent.stem)
# sorted(f.parent.glob('**/*'))
config = json.load((f.parent / 'config.json').open())
print(config)
log = (f.parent / 'log.txt').open().read()
print(log[-300:])

## From txt: old

In [4]:
1/0

In [6]:
# look for log.txt
f = '../deleteme.txt'
text_log = open(f).read()

In [17]:
# just extract the key metric from table 3

# Regular expression to match the table row
table_row_pattern = re.compile(r'(.+)\nTable 3⭐')

# Find the table row
match = table_row_pattern.findall(text_log)
header = """| acc_inc/eval_ds [pp]   |   train |   test |   oos |    rnd |
|:-----------------------|--------:|-------:|------:|-------:|"""
s = header + '\n'.join(match)
print(s)